# making hyperstacks by iterating through a pandas with its dimensions
since there are issuses with reding metadata form the tiff files I calculated and stored number of volumes, z number height and width of each recording in a dataframe.
https://forum.image.sc/t/ome-tiff-z-stack-to-bigtiff/50964/7

In [ ]:
import tifffile as tiff
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import re
from imutils.src import imfunctions
import os
from natsort import natsorted
import math
import glob

## creating the dataframe

In [ ]:
#getting file names
path='/scratch/zimmer/DanielM/ZIm443/flurescence_btiffs/cam1/'
for file in natsorted(os.listdir(path)):
    if file.endswith('.btf'):
        before,sep,after=file.partition('-channel')
        print(before)

In [ ]:
#read in csv with filenames and z number which i got from onenote, as well as h and w which are constant
filename='/groups/zimmer/Daniel_Mitic/code/imutils/imutils/zim443_dimensions.csv'
df=pd.read_csv(filename,sep=';')

In [ ]:
#getting the number of volumes
path='/scratch/zimmer/DanielM/ZIm443/flurescence_btiffs/cam1/'

In [ ]:
#volumes=len of pages/z_stacks
#some of them are not integers: if z is an uneven number the last volume is not gonna have a full amount of stacks. this is why i round it up with math(ceil)
for i,file in enumerate(natsorted(os.listdir(path))):
    if 'camera' in file:
        with tiff.TiffFile(os.path.join(path,file)) as tif:
            print(math.ceil(len(tif.pages)/df.iloc[i,2]))

In [ ]:
#i inserted the number of volumes manually in the csv, so i read it in again here
filename='/groups/zimmer/Daniel_Mitic/code/imutils/imutils/zim443_dimensions.csv'
df=pd.read_csv(filename,sep=';')

# create the hyperstacks

In [ ]:
#inputs  make_hyperstack_from_ometif function
path='/scratch/zimmer/DanielM/ZIm443/cam1/' #path with the folders containing the ome.tif
output_file='/scratch/zimmer/DanielM/hyperstack_daniel/'
dtype='uint16'
imagej=True,
metadata={'axes': 'TZYX'}

In [ ]:
for i,file in enumerate(natsorted(glob.glob(path+'*camera*'))): 
    before,sep,name=file.partition('cam1/') #here i grab the name of the file
    output=os.path.join(output_file+name+'.btf') #making sure the output files have different names
    input_path=os.path.join(path+name+'/') #here i specify the input path as the subfolders which contain the ome.tifs
    shape=tuple(df.iloc[i,1:5]) #iterate through the dataframe to get the corresponding shape for each recording
    imfunctions.make_hyperstack_from_ometif(input_path,output,shape,dtype,imagej,metadata)
print('done')